# Turkish Military Chiefs of Staff - Birthplace Analysis

This notebook analyzes the geographic distribution of birthplaces of Turkish Armed Forces Chiefs of Staff during the Republic era (1920-present).

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## Load Data

In [ ]:
# Load datasets
chiefs_df = pd.read_csv('data/chiefs_of_staff.csv')
coords_df = pd.read_csv('data/city_coordinates.csv')

# Merge data
df = chiefs_df.merge(coords_df, left_on='birthplace', right_on='city', how='left')

print(f"Total Chiefs: {len(df)}")
print(f"Unique Birthplaces: {df['birthplace'].nunique()}")
df.head(10)

## Birthplace Distribution

In [ ]:
birthplace_counts = df['birthplace'].value_counts()
print(birthplace_counts)

plt.figure(figsize=(14, 8))
birthplace_counts.plot(kind='barh', color='steelblue')
plt.title('Distribution of Birthplaces - Turkish Military Chiefs of Staff', fontsize=16, fontweight='bold')
plt.xlabel('Number of Chiefs', fontsize=12)
plt.ylabel('Birthplace', fontsize=12)
plt.tight_layout()
plt.show()

## Regional Analysis

In [ ]:
regional_dist = df.groupby('region')['name'].count().sort_values(ascending=False)
print("\nRegional Distribution:")
print(regional_dist)

plt.figure(figsize=(10, 6))
regional_dist.plot(kind='bar', color='coral')
plt.title('Chiefs by Region', fontsize=14, fontweight='bold')
plt.xlabel('Region', fontsize=12)
plt.ylabel('Number of Chiefs', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Timeline Analysis

In [ ]:
df['start_year'] = pd.to_datetime(df['start_date']).dt.year

plt.figure(figsize=(14, 8))
for city in birthplace_counts.index[:10]:
    city_data = df[df['birthplace'] == city]
    plt.scatter(city_data['start_year'], [city] * len(city_data), s=100, alpha=0.6, label=city)

plt.title('Timeline of Chiefs of Staff by Birthplace', fontsize=16, fontweight='bold')
plt.xlabel('Year Started Service', fontsize=12)
plt.ylabel('Birthplace', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Branch Analysis

In [ ]:
branch_counts = df['branch'].value_counts()
print("\nBranch Distribution:")
print(branch_counts)

plt.figure(figsize=(10, 6))
branch_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Chiefs by Military Branch', fontsize=14, fontweight='bold')
plt.ylabel('')
plt.tight_layout()
plt.show()

## Interactive Map

In [ ]:
# Create interactive map
turkey_center = [39.0, 35.0]
m = folium.Map(location=turkey_center, zoom_start=6, tiles='OpenStreetMap')

marker_cluster = MarkerCluster().add_to(m)

birthplace_data = birthplace_counts.reset_index()
birthplace_data.columns = ['birthplace', 'count']
map_data = coords_df.merge(birthplace_data, left_on='city', right_on='birthplace', how='inner')

for idx, row in map_data.iterrows():
    chiefs_from_city = df[df['birthplace'] == row['city']]['name'].tolist()
    
    popup_text = f"""
    <div style="font-family: Arial; width: 250px;">
        <h4>{row['city']}</h4>
        <p><b>Number of Chiefs:</b> {row['count']}</p>
        <p><b>Region:</b> {row['region']}</p>
        <hr>
        <p><b>Chiefs:</b></p>
        <ul>
    """
    for chief in chiefs_from_city:
        popup_text += f"<li>{chief}</li>"
    popup_text += "</ul></div>"
    
    if row['count'] >= 5:
        color = 'red'
        radius = 15
    elif row['count'] >= 3:
        color = 'orange'
        radius = 12
    elif row['count'] >= 2:
        color = 'blue'
        radius = 10
    else:
        color = 'green'
        radius = 8
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        popup=folium.Popup(popup_text, max_width=300),
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7,
        weight=2
    ).add_to(marker_cluster)

m

## Summary Statistics

In [ ]:
print("=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)
print(f"Total Chiefs of Staff: {len(df)}")
print(f"Unique Birthplaces: {df['birthplace'].nunique()}")
print(f"\nMost Common Birthplace: {birthplace_counts.index[0]} ({birthplace_counts.iloc[0]} chiefs)")
print(f"Most Common Branch: {branch_counts.index[0]} ({branch_counts.iloc[0]} chiefs)")
print(f"\nTime Period: {df['start_year'].min()} - {df['start_year'].max()}")
print("=" * 60)